# Simluate operation of prior system design

In [1]:
import os
os.chdir('..') # move to the root directory (from dev)

In [2]:
import json
import numpy as np

from utils import data_handling, generate_temp_building_files
from utils.plotting import init_profile_fig, add_profile
from utils import build_schema
from energy_system import evaluate_system

In [3]:
from experiments.configs.config import *

Set parameters of case to investigate

In [4]:
n_buildings = 5
case = 'unconstr'

Load sample prior scenarios

In [5]:
scenarios_path = os.path.join(results_dir,f'sampled_scenarios_{n_buildings}b.csv')
scenarios,_ = data_handling.load_scenarios(scenarios_path)

Load prior system design

In [6]:
prior_design_file = os.path.join(results_dir,'prior',f'{case}_{n_buildings}b_design_results.csv')
prior_design = data_handling.load_design_results(prior_design_file)

In [ ]:
print(prior_design)

Retrieve scenario to simulate

In [ ]:
sim_scenario = scenarios[42]
print(sim_scenario)

Build schema for scenario

In [ ]:
with open(os.path.join('resources','base_system_params.json')) as jfile:
        base_params = json.load(jfile)

base_params['data_dir_path'] = dataset_dir

# Set system design parameters
base_params['battery_efficiencies'] = [base_params['base_battery_efficiency']]*n_buildings
base_params.pop('base_battery_efficiency', None)
base_params['battery_energy_capacities'] = prior_design['battery_capacities'].flatten()
base_params['battery_power_capacities'] = [energy*cost_dict['battery_power_ratio'] for energy in prior_design['battery_capacities'].flatten()]
base_params['pv_power_capacities'] = prior_design['solar_capacities'].flatten()

# Get building files, generating temporary files if required (i.e. if scaling needed)
load_files = generate_temp_building_files(sim_scenario, dataset_dir, solar_fname_pattern, building_fname_pattern, 0)

# Build schema for each scenario
params = base_params.copy()
params['building_names'] = [f'TB{i}' for i in range(len(sim_scenario))]
params['load_data_paths'] = [building_fname_pattern.format(id=int(b),year=int(y)) for (b,y,_,_) in sim_scenario]
params['schema_name'] = f'example_prior_sim_schema'
schema_path = build_schema(**params)

Simulate system for selected scenario and create plot

In [ ]:
eval_results = evaluate_system(
    schema_path,
    cost_dict,
    prior_design['grid_con_capacity'],
    design=True,
    show_progress=True,
    plot=True
)

In [11]:
# Clean up temp files
os.remove(schema_path)
for load_file in load_files:
    if 'temp' in load_file:
        os.remove(os.path.join(dataset_dir,load_file))